<a href="https://colab.research.google.com/github/JuanJoMontilla/Senales-y-Sistemas/blob/main/Otros/4p.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit -q
!pip install pyngrok
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile
!pip install -U yt-dlp
!pip install yt-dlp soundfile matplotlib streamlit numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
Using cached requests-2.

In [13]:
%%writefile app.py

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import tempfile
from scipy.fftpack import fft
from scipy.signal import square
from scipy.interpolate import interp1d
import requests
from io import BytesIO
import yt_dlp as youtube_dl
import subprocess
import os
import shutil

# Configuración inicial de la página
st.set_page_config(page_title="Parcial 2 - Fourier SyS", page_icon="📊", layout="wide")

# Barra lateral para seleccionar el punto del parcial y mostrar información general
with st.sidebar:
    st.image("https://lh4.googleusercontent.com/proxy/WNtyuTbDjnnITJFxg1dlI63L0jfIMRf0CIKg75VavFd3ameUuokpEiXIZvafO0UbA3rGKkhjDZ2HFtRWcGiPIn7Syd37PqnCrQuXFNHguRRPYm__safRJi9Q", width=150)
    st.title("Parcial 2 - Fourier SyS")
    st.subheader("Universidad Nacional de Colombia - Sede Manizales")
    st.write("**Señales y Sistemas - 2024-1**")
    st.write("**Integrantes:**")
    st.write("- Juan Felipe Posada Niasa")
    st.write("- Juan Jose Montilla Bastidas")

    # Menú desplegable para seleccionar el punto del parcial
    punto = st.selectbox("Selecciona el punto del parcial",
                         ("Punto 4: THD y Factor de Potencia"))
if punto == "Punto 4: THD y Factor de Potencia":
    st.write("### Punto 4: THD y Factor de Potencia")
    st.markdown("""
**1. Consulte en que consiste la distorcion total de armonicos(Total Harmonic Distortion-(THD)) y el factor de potencia en un circuito electrico.**

**THD:**
Consiste en que se parte de un sistema ideal en el que la función de transferencia es lineal e invariante en el tiempo.
Cuando una señal sinusoidal de frecuencia ω pasa por un dispositivo no ideal y no lineal, se añade contenido adicional en múltiplos nω (armónicos) de la frecuencia original.
La THD es una medida de ese contenido de señal adicional no presente en la señal de entrada. Esta puede ser THD indicándonos con respecto al voltaje o también puede ser THDi, la cual es con respecto a la corriente.

**FP:**
Es la relación entre la potencia real y la potencia aparente en un circuito o sistema de distribución. La potencia real se mide en W o kW, y es la que utilizan los motores, las luces y otros dispositivos para producir trabajo útil. La potencia aparente se mide en VA o kVA, que es la suma vectorial de la potencia real (trabajo útil), la potencia reactiva (almacenada y liberada por inductores y condensadores y muestra el desplazamiento de fase entre las formas de onda de corriente y tensión), y la potencia armónica (la cual se pierde por distorsión armónica).
""")
    st.markdown("""
**2. ¿Como puede calcularse el THD desde la FFT?**

**Los pasos a seguir serían:**

1) **Realizar la FFT:** Esto transforma la señal del dominio del tiempo al dominio de la frecuencia, permitiendo identificar las diferentes componentes de frecuencia, incluyendo la fundamental y los armónicos.

2) **Identificar la frecuencia fundamental:** Una vez que se tiene el espectro de frecuencias, se debe identificar la frecuencia fundamental $V_1$, que es la frecuencia más baja y representa la señal principal.

3) **Calcular las amplitudes de los armónicos:** A partir del espectro de la FFT, se deben extraer las amplitudes de los armónicos $V_n$, donde $n$ es el número del armónico, comenzando desde el armónico 2 y tomando los componentes de frecuencia que son múltiplos enteros de la frecuencia fundamental.

4) **Aplicar la fórmula del THD:**

$$
THD = \\frac{\\sqrt{V_2^2 + V_3^2 + \\cdots + V_n^2}}{V_1}
$$

donde $V_n$ es el valor del $n$-ésimo armónico de la señal de salida del circuito (en términos de tensión), y $V_1$ es la componente fundamental.
""")
    st.markdown("""
**3. ¿Como puede calcularse la distorsión del factor de potencia con base al THD?**

El factor de potencia de distorsión es la relación entre la potencia real y la potencia aparente debido a la distorsión armónica total (THD). No se pueden añadir condensadores a un circuito para compensar el factor de potencia de distorsión. La impedancia de los condensadores disminuye con la frecuencia. Por lo tanto, un condensador puede convertirse en un sumidero de armónicos de alta frecuencia. Se utilizan tipos especiales de transformadores o filtros armónicos sintonizados que constan de condensadores e inductores para corregir el factor de potencia de distorsión.

El factor de potencia de distorsión se calcula de la siguiente manera:

$$
PF_{THD} = \\sqrt{\\frac{1}{1 + (THD)^2}}
$$
""")
    st.write("### Cálculo THD y Distorsión del Factor de Potencia")
    st.markdown("""
**i)Carga Netamente Resistiva**
""")
    ###########################################33
    st.markdown("""
**ii)Carga Netamente Resistiva**
""")

Overwriting app.py


In [14]:
from pyngrok import ngrok

# Token personal de ngrok
token = "2lLFMBWiinIoZUi3bX7RF26eFeD_5KBTy8DqrMYoE2mf5Sqnc"
ngrok.set_auth_token(token)

# Ejecutar Streamlit en el puerto 5011
!nohup streamlit run app.py --server.port 5011 &

# Iniciar el túnel Ngrok y exponer Streamlit en una URL.
public_url = ngrok.connect(5011)
print(f"Aplicación aquí: {public_url}")

nohup: appending output to 'nohup.out'
Aplicación aquí: NgrokTunnel: "https://9e34-34-82-69-97.ngrok-free.app" -> "http://localhost:5011"
